In [46]:
from hmmlearn.hmm import GaussianHMM
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

In [47]:
df=pd.read_csv("HD.csv",encoding= 'unicode_escape')
df["Returns"] = df["Adj Close"].pct_change()
df["Intraday Volatility"] = df["High"] - df["Low"]
df["Fractional Change"] = (df["Close"] - df["Open"]) / df["Open"]
df["Fractional High"] = (df["High"] - df["Open"]) / df["Open"]
df["Fractional Low"] = (df["Open"] - df["Low"]) / df["Open"]
df["Returns"] = np.nan_to_num(df["Returns"]) 

In [48]:
rets1 = np.column_stack([df['Returns']])
hmm_model = GaussianHMM(n_components=2, covariance_type="diag", n_iter=1000).fit(rets1)
print("Model Score:", hmm_model.score(rets1))
print(hmm_model.transmat_ * 100)

Model Score: 33.94008234418075
[[8.23523929e-05 9.99999176e+01]
 [3.33443051e-05 9.99999667e+01]]


In [49]:
scaler = StandardScaler()
rets2 = scaler.fit_transform(df[['Returns','Volume']])
hmm_model = GaussianHMM(n_components=2, covariance_type="spherical", n_iter=1000).fit(rets2)
print("Model Score:", hmm_model.score(rets2))
print(hmm_model.transmat_ * 100)

Model Score: -36.462100097337874
[[49.95998135 50.04001865]
 [ 8.34157451 91.65842549]]


In [50]:
scaler = StandardScaler()
rets3 = scaler.fit_transform(df[['Returns', 'Volume', 'Intraday Volatility']])
hmm_model = GaussianHMM(n_components=2, covariance_type="full", n_iter=1000).fit(rets3)
print("Model Score:", hmm_model.score(rets3))
print(hmm_model.transmat_ * 100)

Model Score: -2.674922113358601
[[91.66666667  8.33333333]
 [50.         50.        ]]


In [51]:
rets4 = df[['Fractional Change', 'Fractional High', 'Fractional Low']]
hmm_model = GaussianHMM(n_components=2, covariance_type="full", n_iter=1000).fit(rets4)
print("Model Score:", hmm_model.score(rets4))
print(hmm_model.transmat_ * 100)

Model Score: 127.81210019434542
[[ 80.35841614  19.64158386]
 [100.           0.        ]]


In [52]:
import sklearn.mixture as mix
import scipy.stats as scs
import pandas_datareader.data as web
model = mix.GaussianMixture(n_components=3,covariance_type="full",n_init=100,random_state=7).fit(rets2)
hidden_states = model.predict(rets2)

In [53]:
print("Means and vars of each hidden state")
for i in range(model.n_components):
    print("{0}th hidden state".format(i))
    print("mean = ", model.means_[i])
    print("var = ", np.diag(model.covariances_[i]))
    print()

Means and vars of each hidden state
0th hidden state
mean =  [-0.0328109  -0.41404604]
var =  [0.33529728 0.14721326]

1th hidden state
mean =  [-1.1070441  2.3453241]
var =  [0.85585746 0.04896214]

2th hidden state
mean =  [2.60781898 0.27790431]
var =  [1.e-06 1.e-06]



In [54]:
model = mix.GaussianMixture(n_components=3,covariance_type="full",n_init=100,random_state=7).fit(rets1)
hidden_states1 = model.predict(rets1)
hidden_states1

array([1, 1, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [55]:
model = mix.GaussianMixture(n_components=3,covariance_type="full",n_init=100,random_state=7).fit(rets2)
hidden_states2 = model.predict(rets2)
hidden_states2

array([0, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [56]:
model = mix.GaussianMixture(n_components=3,covariance_type="full",n_init=100,random_state=7).fit(rets3)
hidden_states3 = model.predict(rets3)
hidden_states3

array([0, 2, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [57]:
model = mix.GaussianMixture(n_components=3,covariance_type="full",n_init=100,random_state=7).fit(rets4)
hidden_states4 = model.predict(rets4)
hidden_states4

array([0, 1, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [58]:
df1 = pd.DataFrame({ 'hidden_states1': hidden_states1 })

In [59]:
#df2['hidden_states2']=pd.Series(hidden_states2)

In [60]:
df2 = pd.DataFrame({ 'hidden_states2': hidden_states2 })

In [61]:
df3 = pd.DataFrame({ 'hidden_states3': hidden_states3 })

In [62]:
df4 = pd.DataFrame({ 'hidden_states4': hidden_states4 })

In [63]:
#df1['hidden_states3']=pd.Series(hidden_states3)

In [64]:
#df1['hidden_states4']=pd.Series(hidden_states4)

In [65]:
df1.to_csv('HDHMM1.csv',index=False)

In [66]:
df2.to_csv('HDHMM2.csv',index=False)

In [67]:
df3.to_csv('HDHMM3.csv',index=False)

In [68]:
df4.to_csv('HDHMM4.csv',index=False)